In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision.models.segmentation import deeplabv3_resnet101, deeplabv3_mobilenet_v3_large, deeplabv3_resnet50
from torchvision.models.segmentation.deeplabv3 import DeepLabV3_ResNet101_Weights, DeepLabV3_MobileNet_V3_Large_Weights, DeepLabV3_ResNet50_Weights
import time
import os
import copy
import csv
from sklearn.metrics import f1_score, roc_auc_score
from torchvision.models.segmentation.deeplabv3 import DeepLabHead

from load_data import SegmentationDataset
import torchvision.transforms as T
import random
from torch.utils.data import DataLoader, Subset, random_split
from pathlib import Path


In [2]:
if torch.cuda.is_available():
    device = 'cuda'
# elif torch.backends.mps.is_available():
#     device = 'mps'
else:
    device = 'cpu'

print(f'Device: {device}')

Device: cpu


In [3]:
weights = 'resnet50'

if weights == 'resnet50':
    model_weights = DeepLabV3_ResNet50_Weights.DEFAULT
    model = deeplabv3_resnet50(weights='DEFAULT').to(device)
elif weights == 'resnet101':
    model_weights = DeepLabV3_ResNet101_Weights.DEFAULT
    model = deeplabv3_resnet101(weights=model_weights).to(device)
elif weights == 'mobilenet':
    model_weights = DeepLabV3_MobileNet_V3_Large_Weights.DEFAULT
    model = deeplabv3_mobilenet_v3_large(weights=model_weights).to(device)
else:
    raise NameError('Chosen weights not available')


In [4]:
num_classes = 2

model.classifier[4] = nn.Conv2d(256, num_classes, 1)
model.aux_classifier[4] = nn.Conv2d(256, num_classes, 1)

In [5]:
print(model)

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [6]:
def train_model(model, dataloader, bpath, num_epochs, device):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    # Use gpu if available
    model.to(device)

    criterion = torch.nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    metrics = {'f1_score': f1_score, 'auroc': roc_auc_score}
    
    # Initialize the log file for training and testing loss and metrics
    fieldnames = ['epoch', 'Train_loss', 'Test_loss'] + \
        [f'Train_{m}' for m in metrics.keys()] + \
        [f'Test_{m}' for m in metrics.keys()]
    
    if not bpath.exists():
        bpath.mkdir()
    with open(os.path.join(bpath, 'log.csv'), 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

    for epoch in range(1, num_epochs + 1):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)
        # Each epoch has a training and validation phase
        # Initialize batch summary
        batchsummary = {a: [0] for a in fieldnames}

        for phase in ['Train', 'Test']:
            if phase == 'Train':
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            # Iterate over data.
            for images, masks in dataloader:
                images = images.to(device)
                masks = masks.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # track history if only in train
                with torch.set_grad_enabled(phase == 'Train'):
                    outputs = model(images)
                    print(outputs)
                    loss = criterion(outputs['out'], masks)
                    y_pred = outputs['out'].data.cpu().numpy().ravel()
                    y_true = masks.data.cpu().numpy().ravel()
                    for name, metric in metrics.items():
                        if name == 'f1_score':
                            # Use a classification threshold of 0.1
                            batchsummary[f'{phase}_{name}'].append(
                                metric(y_true > 0, y_pred > 0.1))
                        else:
                            batchsummary[f'{phase}_{name}'].append(
                                metric(y_true.astype('uint8'), y_pred))

                    # backward + optimize only if in training phase
                    loss.backward()
                    optimizer.step()
            batchsummary['epoch'] = epoch
            epoch_loss = loss
            batchsummary[f'{phase}_loss'] = epoch_loss.item()
            print('{} Loss: {:.4f}'.format(phase, loss))
        for field in fieldnames[3:]:
            batchsummary[field] = np.mean(batchsummary[field])
        print(batchsummary)
        with open(os.path.join(bpath, 'log.csv'), 'a', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(batchsummary)
            # deep copy the model
            if phase == 'Test' and loss < best_loss:
                best_loss = loss
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Lowest Loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [7]:
percent = 0.1

transform = T.Compose([
        # transforms.Resize((256, 256)),
        T.ToTensor(),
        # T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

dataset = SegmentationDataset(
        root_dir=os.path.join('data','segmentation_dataset'),
        transform=transform
    )

# Split the dataset into train, validation, and test
total_count = len(dataset)
train_count = int(0.8 * total_count)
valid_count = int(0.10 * total_count)
test_count = total_count - train_count - valid_count

train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_count, valid_count, test_count])

if percent < 1:
    subset_size = int(percent*len(train_dataset))
    indices = list(range(len(train_dataset)))
    random_indices = random.sample(indices, subset_size)
    train_dataset = Subset(train_dataset, random_indices)

# Create data loaders for each split
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


In [8]:
trained_model = train_model(model, train_loader, Path('./test'), 1, device)

Epoch 1/1
----------
OrderedDict([('out', tensor([[[[-0.3309, -0.3309, -0.3309,  ..., -0.0749, -0.0749, -0.0749],
          [-0.3309, -0.3309, -0.3309,  ..., -0.0749, -0.0749, -0.0749],
          [-0.3309, -0.3309, -0.3309,  ..., -0.0749, -0.0749, -0.0749],
          ...,
          [ 0.1580,  0.1580,  0.1580,  ..., -0.0064, -0.0064, -0.0064],
          [ 0.1580,  0.1580,  0.1580,  ..., -0.0064, -0.0064, -0.0064],
          [ 0.1580,  0.1580,  0.1580,  ..., -0.0064, -0.0064, -0.0064]]],


        [[[-0.3378, -0.3378, -0.3378,  ..., -0.3299, -0.3299, -0.3299],
          [-0.3378, -0.3378, -0.3378,  ..., -0.3299, -0.3299, -0.3299],
          [-0.3378, -0.3378, -0.3378,  ..., -0.3299, -0.3299, -0.3299],
          ...,
          [ 0.1262,  0.1262,  0.1262,  ..., -0.0254, -0.0254, -0.0254],
          [ 0.1262,  0.1262,  0.1262,  ..., -0.0254, -0.0254, -0.0254],
          [ 0.1262,  0.1262,  0.1262,  ..., -0.0254, -0.0254, -0.0254]]]],
       grad_fn=<UpsampleBilinear2DBackward0>)), ('aux', t